# face4text 
### Search a zipfile of images to find which images contain the input text and detect faces in that image
#### We use tesseract and opencv for the task. The images should be stored in a zip file


In [ ]:
import pytesseract
from PIL import Image
import cv2 as cv
import numpy as np
import zipfile

In [ ]:
#Test
print(pytesseract.image_to_string(Image.open('a-0.png')))

In [ ]:
# loading the face detection classifier
face_cascade = cv.CascadeClassifier(cv.data.haarcascades + 'haarcascade_frontalface_default.xml')

#### Replace 'images.zip' with your zipfile location

In [ ]:
#Create data[] by doing OCR
def create_data_list():
    with zipfile.ZipFile('images.zip') as zf:
        for entry in zf.infolist():
            with zf.open(entry) as file:
                print("Loading Page")

                image = Image.open(file)
                image_data = {}

                #Storing the color image in image_data{}
                image_data['image'] = image
                image = image.convert('1')
                new_size = (int(image.width*2), int(image.height*2))
                image = image.resize(new_size, Image.ANTIALIAS)

                #OCR

                print("Doing OCR")
                text = pytesseract.image_to_string(image)
                image_data['text'] = text

                data.append(image_data)

In [ ]:
#Test
create_data_list()
print(data[0]['image'])

In [ ]:
#Do face detection and generate the faces list of an image
def generate_faces_list(pil_image):
    #Return a list of cropped faces when PIL image is passed
    pil_image_ref = pil_image
    ocv_image = np.array(pil_image)
    
    gray = cv.cvtColor(ocv_image, cv.COLOR_BGR2GRAY)
    print(gray)
    faces = face_cascade.detectMultiScale(gray, minNeighbors=15)
    print('face detection done...')
    
    
    cropped_faces = []

    for x,y,w,h in faces:
        cropped_faces.append(pil_image_ref.crop((x,y,x+w,y+h)))
    
    return cropped_faces

In [ ]:
#Return a list of page numbers where the text contains the string that is passed
def find_pages_text(text):
     
    page_num = []
    
    for x in range(len(data)):
        print("checking page {}".format(x))
        if text in data[x]['text']:
            page_num.append(x)
            
    return page_num

In [ ]:
#Display the faces
def display_faces(faces):
    #Display the faces as a single image
    thumb_size = (100,100)
    thumbnails = []
    if faces == []:
        print("But there were no faces in that file!")
    else:
        for image in faces:
            image.thumbnail(thumb_size)
            thumbnails.append(image)

        first_image = thumbnails[0]
        rows = len(thumbnails)/5+1
        contact_sheet = Image.new(first_image.mode, (first_image.width*5,int(first_image.height*rows)))
        
        x = 0
        y = 0
        
        for img in thumbnails:
            contact_sheet.paste(img, (x, y) )
        
            if x+first_image.width == contact_sheet.width:
                x=0
                y=y+first_image.height
            else:
                x=x+first_image.width
        display(contact_sheet)

In [ ]:
#Main 

#Data - List of Dicts [{'image':<image>, 'text':'text'}]
data = []

search_text = input("Enter Text :") #Input the Text to search for

create_data_list()

In [ ]:
#Test
print(data[0]['image'])

In [ ]:
pages_to_face_detection = find_pages_text(search_text)

In [ ]:
#Test
print(pages_to_face_detection)

In [ ]:
for x in pages_to_face_detection:
    print('Results found in file a-{}.png'.format(x))
    faces = []
    for image in generate_faces_list(data[x]['image']):
        faces.append(image)
            
    display_faces(faces)